# Supervised Machine Learning & OpenAI: End-to-End Workflow

## 1. Dataset Preparation for Fine-Tuning (Code Generation Task)

In [1]:
!pip install pandas

import pandas as pd

# Example dataset for fine-tuning code generation (input: prompt, output: code)
data = [
    {
        "prompt": "Write a Python function to add two numbers.",
        "completion": "def add(a, b):\n    return a + b"
    },
    {
        "prompt": "Write a Python function to check if a number is even.",
        "completion": "def is_even(n):\n    return n % 2 == 0"
    },
    {
        "prompt": "Write a Python function to find factorial.",
        "completion": "def factorial(n):\n    return 1 if n == 0 else n * factorial(n-1)"
    },
]

df = pd.DataFrame(data)
df.to_csv("code_generation_dataset.csv", index=False)
df.head()


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


,prompt,completion
0,Write a Python function to add two numbers.,"def add(a, b):\n return a + b"
1,Write a Python function to check if a number i...,def is_even(n):\n return n % 2 == 0
2,Write a Python function to find factorial.,def factorial(n):\n return 1 if n == 0 else...


## 2. Supervised Model: Train-Test Split and Fine-Tuning Prep
For OpenAI fine-tuning, you typically format your data as JSONL. Let's convert and show a basic split.

In [2]:
import json

# Train-test split (80-20 for demo)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Save prompts and completions as simpler JSONL (not OpenAI format)
def save_jsonl_simple(df, filename):
    with open(filename, "w") as f:
        for _, row in df.iterrows():
            entry = {
                "prompt": row["prompt"],
                "completion": row["completion"]
            }
            f.write(json.dumps(entry) + "\n")

save_jsonl_simple(train_df, "train.jsonl")
save_jsonl_simple(test_df, "test.jsonl")

In [3]:
!pip install google-generativeai python-dotenv --quiet
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### A. Prompt Engineering for Code Generation

In [4]:
def generate_code(prompt):
    model = genai.GenerativeModel("gemini-1.5-flash")  
    response = model.generate_content(prompt)
    return response.text

### B. Code Optimization Using OpenAI

In [5]:
raw_code = """
def add_numbers(a, b):
    result = 0
    result = a + b
    return result
"""

opt_prompt = f"Optimize the following Python code for simplicity and efficiency:\n\n{raw_code}"

optimized_code = generate_code(opt_prompt)
print("Optimized Code:\n", optimized_code)

Optimized Code:
 The simplest and most efficient way to write this function is:

```python
def add_numbers(a, b):
  return a + b
```

The original code created an unnecessary intermediate variable `result`.  Python's direct assignment and return statement handle this operation perfectly without extra overhead.



### C. Prompt Engineering for Debugging

In [6]:
buggy_code = """
def divide(a, b):
    return a / b
print(divide(10, 0))
"""

debug_prompt = f"Find and fix the bug in the following Python code:\n\n{buggy_code}"

fixed_code = generate_code(debug_prompt)
print("Fixed Code:\n", fixed_code)

Fixed Code:
 The bug is a `ZeroDivisionError`.  Dividing by zero is undefined in mathematics and causes an exception in Python.

To fix it, you need to add error handling.  Here are a couple of ways:

**Method 1: Using a `try-except` block:**

```python
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return "Division by zero is not allowed."  # Or some other appropriate action

print(divide(10, 0))  # Output: Division by zero is not allowed.
print(divide(10, 2))  # Output: 5.0
```

This method attempts the division. If a `ZeroDivisionError` occurs, it catches the exception and returns a user-friendly message.


**Method 2: Checking for zero before division:**

```python
def divide(a, b):
    if b == 0:
        return "Division by zero is not allowed." # Or return None, float('inf'), or raise an exception
    else:
        return a / b

print(divide(10, 0))  # Output: Division by zero is not allowed.
print(divide(10, 2))  # Output: 5.0
```

This met

## 4. [Optional] Evaluate Generated Code with Automated Testing
You can run generated code snippets using exec, but always sanitize untrusted code. For demonstration:

In [7]:
import re
def extract_code_block(text):
    """
    Extracts the first python code block from LLM output.
    If no code block, returns raw text.
    """
    code_blocks = re.findall(r"```(?:python)?(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if code_blocks:
        return code_blocks[0].strip()
    return text.strip()

def test_reverse_string():
    prompt = "Write a Python function named `reverse_string` that takes a string and returns it reversed."
    code_response = generate_code(prompt)
    print("LLM Response:\n", code_response)
    code = extract_code_block(code_response)
    print("Extracted Code:\n", code)
    exec(code, globals())  # Define function in notebook/global scope
    try:
        assert reverse_string("OpenAI") == "IAnepO"
        assert reverse_string("") == ""
        assert reverse_string("a") == "a"
        print("Test passed!")
    except Exception as e:
        print("Test failed:", e)

test_reverse_string()

LLM Response:
 ```python
def reverse_string(input_string):
  """Reverses a given string.

  Args:
    input_string: The string to be reversed.

  Returns:
    The reversed string.  Returns an empty string if the input is None or not a string.
  """
  if not isinstance(input_string, str) or input_string is None:
    return ""
  return input_string[::-1]

```
Extracted Code:
 def reverse_string(input_string):
  """Reverses a given string.

  Args:
    input_string: The string to be reversed.

  Returns:
    The reversed string.  Returns an empty string if the input is None or not a string.
  """
  if not isinstance(input_string, str) or input_string is None:
    return ""
  return input_string[::-1]
Test passed!


## Summary
*Dataset Preparation:* Created and exported a dataset for code generation tasks.

*Modeling & Fine-Tuning:* Demonstrated how to prepare data for OpenAI fine-tuning.

*Prompt Engineering:* Showed prompts for code generation, code optimization, and debugging.

*OpenAI API Usage:* Used the OpenAI API for advanced code assistance.